In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [3]:
covidx_dir = '/media/ubuntu/MyHDataStor3/datasets/COV19D/'
covidx_img_dir= covidx_dir 

print(covidx_dir)
print(covidx_img_dir)

/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/


In [4]:
def seedfill(img, val):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), val);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), val);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), val);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), val);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    contours, hierarchy   = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(4, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1

    
    bin_image2 = 1 - bin_image 
    seedfill(bin_image2,1)
  

    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(bin_image2, cmap=plt.cm.bone)     
    
    
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv,255)/255 
    cleared = clear_border(bin_image_inv)
    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(2)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [6]:
#this code is to generate bounding box and percent of lung mask for traning and validation dataset
from tqdm import tqdm

debug = False 

subset = 'test'
ct_scans = glob(covidx_dir+subset+'/*/*')
print(ct_scans[:10])

val_annot_file = covidx_dir +"list_{}.txt".format(subset) 
print(val_annot_file)

fp = open(val_annot_file,'w')

par = tqdm(total=len(ct_scans))
count = 0 
for ct in ct_scans:
    
    par.update() 
    
    sfiles = os.listdir(ct)
    sfiles = [x for x in sfiles if not x.startswith('.')]
    sfiles.sort(key = lambda x: int(x.split('.')[0]))

    for f in sfiles:        
        
        f_full = os.path.join(ct,f)
        
        img = cv2.imread(f_full, cv2.IMREAD_UNCHANGED)
        mask, (xmin, ymin, xmax, ymax) = auto_body_crop(img,plot=False)

        img_crop = img[ymin:ymax,xmin:xmax] 
        mask_crop = mask[ymin:ymax,xmin:xmax] 
        
        img_crop_2 = img_crop.copy()
        #img_crop_2 = 255 - img_crop_2                
        mask_crop_ind = (mask_crop == 0) 
        img_crop_2[mask_crop_ind] = 0 
        img_crop_3 = seedfill(img_crop_2,255)
        
        
        rows,cols = img_crop_3.shape
        percent_dark = np.sum(img_crop_3 <10)/rows/cols 
        #print(percent_dark)
        if debug: 
            fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(5,1,figsize=(5, 40))
            ax1.imshow(img,cmap='gray')                    
            ax2.imshow(img_crop,cmap='gray')
            ax3.imshow(img_crop_2,cmap='gray')
            ax4.imshow(img_crop_3,cmap='gray')               
            ax5.imshow(mask_crop,cmap='gray')               
        
        if 'non-covid' in f_full:
            c = 'non-covid'
        else:
            c = 'covid'

        #print(f_full,c)

            
        fp.write("{} {} {} {} {} {} {:2f}\n".format(f_full,c,xmin,ymin,xmax,ymax,percent_dark))  

        count += 1          

fp.close()


  0%|          | 0/5546 [00:00<?, ?it/s]

['/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1401', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1358', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1165', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1665', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1234', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1189', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1773', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_956', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1302', '/media/ubuntu/MyHDataStor3/datasets/COV19D/test/part1_0_1_test_set_ECCV/ct_scan_1354']
/media/ubuntu/MyHDataStor3/datasets/COV19D/list_test.txt


100%|██████████| 5546/5546 [57:17<00:00,  1.84it/s]  

In [8]:
#this code is to generate bounding box and percent of lung mask for traning and validation dataset
from tqdm import tqdm

debug = False 

subset = 'train'

ct_scans = glob(covidx_dir+subset+'/*/*/*')
print(ct_scans[:10])

val_annot_file = covidx_dir +"list_{}.txt".format(subset) 
print(val_annot_file)

fp = open(val_annot_file,'w')

par = tqdm(total=len(ct_scans))
count = 0 

for ct in ct_scans:
    
    par.update() 
    
    sfiles = os.listdir(ct)
    sfiles = [x for x in sfiles if not x.startswith('.')]
    sfiles.sort(key = lambda x: int(x.split('.')[0]))

    for f in sfiles:        
        
        f_full = os.path.join(ct,f)
        
        img = cv2.imread(f_full, cv2.IMREAD_UNCHANGED)
        mask, (xmin, ymin, xmax, ymax) = auto_body_crop(img,plot=False)

        img_crop = img[ymin:ymax,xmin:xmax] 
        mask_crop = mask[ymin:ymax,xmin:xmax] 
        
        img_crop_2 = img_crop.copy()
        #img_crop_2 = 255 - img_crop_2                
        mask_crop_ind = (mask_crop == 0) 
        img_crop_2[mask_crop_ind] = 0 
        img_crop_3 = seedfill(img_crop_2,255)
        
        
        rows,cols = img_crop_3.shape
        percent_dark = np.sum(img_crop_3 <10)/rows/cols 
        #print(percent_dark)
        if debug: 
            fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(5,1,figsize=(5, 40))
            ax1.imshow(img,cmap='gray')                    
            ax2.imshow(img_crop,cmap='gray')
            ax3.imshow(img_crop_2,cmap='gray')
            ax4.imshow(img_crop_3,cmap='gray')               
            ax5.imshow(mask_crop,cmap='gray')               
        
        
        if 'non-covid' in f_full:
            c = 'non-covid'
        else:
            c = 'covid'
        
        fp.write("{} {} {} {} {} {} {:2f}\n".format(f_full,c,xmin,ymin,xmax,ymax,percent_dark))  

        count += 1          

fp.close()




  0%|          | 0/1992 [00:00<?, ?it/s]

['/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_449', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_362', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_310', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_369', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_232', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_403', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_450', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_297', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_323', '/media/ubuntu/MyHDataStor3/datasets/COV19D/train/covid/subset2/ct_scan_445']
/media/ubuntu/MyHDataStor3/datasets/COV19D/list_train.txt




  0%|          | 2/1992 [00:00<07:04,  4.69it/s]

  0%|          | 3/1992 [00:01<20:11,  1.64it/s]

  0%|          | 4/1992 [00:02<16:22,  2.02it/s]

  0%|          | 5/1992 [00:02<13:04,  2.53it/s]

  0%|          | 6/1992 [00:03<18:42,  1.77it/s]

  0%|          | 7/1992 [00:03<14:49,  2.23it/s]

  0%|          | 8/1992 [00:04<20:33,  1.61it/s]

  0%|          | 9/1992 [00:04<16:40,  1.98it/s]

  1%|          | 10/1992 [00:05<23:35,  1.40it/s]

  1%|          | 11/1992 [00:06<19:53,  1.66it/s]

  1%|          | 12/1992 [00:07<25:09,  1.31it/s]

  1%|          | 13/1992 [00:08<30:06,  1.10it/s]

  1%|          | 14/1992 [00:08<22:40,  1.45it/s]

  1%|          | 15/1992 [00:10<30:42,  1.07it/s]

  1%|          | 16/1992 [00:10<23:09,  1.42it/s]

  1%|          | 17/1992 [00:11<26:16,  1.25it/s]

  1%|          | 18/1992 [00:12<29:08,  1.13it/s]

  1%|          | 19/1992 [00:12<22:02,  1.49it/s]

  1%|          | 20/1992 [00:13<24:03,  1.37it/s]

  1%|          | 21/1992 [00:14<27:52

 14%|█▍        | 288/1992 [04:12<27:14,  1.04it/s]

 15%|█▍        | 289/1992 [04:14<30:58,  1.09s/it]

 15%|█▍        | 290/1992 [04:15<32:47,  1.16s/it]

 15%|█▍        | 291/1992 [04:17<35:33,  1.25s/it]

 15%|█▍        | 292/1992 [04:18<37:43,  1.33s/it]

 15%|█▍        | 293/1992 [04:18<27:46,  1.02it/s]

 15%|█▍        | 294/1992 [04:19<21:48,  1.30it/s]

 15%|█▍        | 295/1992 [04:19<18:34,  1.52it/s]

 15%|█▍        | 296/1992 [04:20<20:37,  1.37it/s]

 15%|█▍        | 297/1992 [04:21<27:11,  1.04it/s]

 15%|█▍        | 298/1992 [04:23<30:05,  1.07s/it]

 15%|█▌        | 299/1992 [04:24<33:06,  1.17s/it]

 15%|█▌        | 300/1992 [04:24<24:17,  1.16it/s]

 15%|█▌        | 301/1992 [04:25<24:37,  1.14it/s]

 15%|█▌        | 302/1992 [04:25<18:24,  1.53it/s]

 15%|█▌        | 303/1992 [04:26<19:24,  1.45it/s]

 15%|█▌        | 304/1992 [04:26<16:53,  1.67it/s]

 15%|█▌        | 305/1992 [04:27<14:33,  1.93it/s]

 15%|█▌        | 306/1992 [04:28<19:25,  1.45it/s]

 15%|█▌     

 29%|██▉       | 573/1992 [08:10<16:21,  1.45it/s]

 29%|██▉       | 574/1992 [08:10<12:46,  1.85it/s]

 29%|██▉       | 575/1992 [08:11<21:18,  1.11it/s]

 29%|██▉       | 576/1992 [08:13<23:24,  1.01it/s]

 29%|██▉       | 577/1992 [08:14<23:22,  1.01it/s]

 29%|██▉       | 579/1992 [08:14<16:57,  1.39it/s]

 29%|██▉       | 580/1992 [08:15<18:48,  1.25it/s]

 29%|██▉       | 581/1992 [08:17<28:25,  1.21s/it]

 29%|██▉       | 582/1992 [08:18<27:12,  1.16s/it]

 29%|██▉       | 583/1992 [08:19<27:21,  1.16s/it]

 29%|██▉       | 584/1992 [08:21<29:11,  1.24s/it]

 29%|██▉       | 585/1992 [08:22<28:57,  1.23s/it]

 29%|██▉       | 586/1992 [08:23<30:22,  1.30s/it]

 29%|██▉       | 587/1992 [08:23<22:19,  1.05it/s]

 30%|██▉       | 588/1992 [08:24<21:30,  1.09it/s]

 30%|██▉       | 589/1992 [08:24<15:52,  1.47it/s]

 30%|██▉       | 590/1992 [08:25<12:39,  1.85it/s]

 30%|██▉       | 591/1992 [08:25<10:07,  2.30it/s]

 30%|██▉       | 592/1992 [08:26<14:48,  1.57it/s]

 30%|██▉    

 43%|████▎     | 857/1992 [11:15<11:19,  1.67it/s]

 43%|████▎     | 858/1992 [11:15<08:39,  2.18it/s]

 43%|████▎     | 859/1992 [11:16<11:54,  1.59it/s]

 43%|████▎     | 860/1992 [11:16<09:53,  1.91it/s]

 43%|████▎     | 861/1992 [11:17<13:33,  1.39it/s]

 43%|████▎     | 862/1992 [11:18<10:40,  1.76it/s]

 43%|████▎     | 863/1992 [11:19<13:49,  1.36it/s]

 43%|████▎     | 864/1992 [11:19<11:23,  1.65it/s]

 43%|████▎     | 865/1992 [11:19<09:40,  1.94it/s]

 43%|████▎     | 866/1992 [11:20<12:44,  1.47it/s]

 44%|████▎     | 867/1992 [11:21<09:58,  1.88it/s]

 44%|████▎     | 868/1992 [11:21<07:45,  2.42it/s]

 44%|████▎     | 869/1992 [11:21<06:06,  3.07it/s]

 44%|████▎     | 870/1992 [11:22<10:55,  1.71it/s]

 44%|████▎     | 871/1992 [11:22<09:24,  1.99it/s]

 44%|████▍     | 872/1992 [11:23<13:05,  1.43it/s]

 44%|████▍     | 873/1992 [11:24<14:11,  1.31it/s]

 44%|████▍     | 874/1992 [11:26<16:22,  1.14it/s]

 44%|████▍     | 875/1992 [11:26<13:20,  1.39it/s]

 44%|████▍  

 57%|█████▋    | 1137/1992 [14:12<05:11,  2.74it/s]

 57%|█████▋    | 1138/1992 [14:12<06:23,  2.23it/s]

 57%|█████▋    | 1140/1992 [14:14<07:15,  1.96it/s]

 57%|█████▋    | 1141/1992 [14:14<06:22,  2.22it/s]

 57%|█████▋    | 1142/1992 [14:14<06:49,  2.08it/s]

 57%|█████▋    | 1143/1992 [14:15<06:08,  2.30it/s]

 57%|█████▋    | 1144/1992 [14:15<05:36,  2.52it/s]

 57%|█████▋    | 1145/1992 [14:16<09:28,  1.49it/s]

 58%|█████▊    | 1146/1992 [14:17<07:12,  1.96it/s]

 58%|█████▊    | 1147/1992 [14:17<06:48,  2.07it/s]

 58%|█████▊    | 1148/1992 [14:18<09:46,  1.44it/s]

 58%|█████▊    | 1149/1992 [14:19<10:22,  1.35it/s]

 58%|█████▊    | 1150/1992 [14:20<10:35,  1.32it/s]

 58%|█████▊    | 1151/1992 [14:21<11:10,  1.25it/s]

 58%|█████▊    | 1152/1992 [14:21<09:12,  1.52it/s]

 58%|█████▊    | 1153/1992 [14:22<10:50,  1.29it/s]

 58%|█████▊    | 1154/1992 [14:24<17:36,  1.26s/it]

 58%|█████▊    | 1155/1992 [14:25<13:25,  1.04it/s]

 58%|█████▊    | 1156/1992 [14:26<14:23,  1.03

 71%|███████   | 1415/1992 [16:56<06:31,  1.47it/s]

 71%|███████   | 1416/1992 [16:56<05:24,  1.78it/s]

 71%|███████   | 1417/1992 [16:56<04:07,  2.32it/s]

 71%|███████   | 1418/1992 [16:57<05:22,  1.78it/s]

 71%|███████   | 1419/1992 [16:58<06:39,  1.43it/s]

 71%|███████▏  | 1420/1992 [16:58<05:11,  1.84it/s]

 71%|███████▏  | 1421/1992 [16:59<06:59,  1.36it/s]

 71%|███████▏  | 1422/1992 [17:01<08:07,  1.17it/s]

 71%|███████▏  | 1423/1992 [17:01<06:18,  1.50it/s]

 71%|███████▏  | 1424/1992 [17:02<09:04,  1.04it/s]

 72%|███████▏  | 1425/1992 [17:04<11:31,  1.22s/it]

 72%|███████▏  | 1426/1992 [17:05<11:29,  1.22s/it]

 72%|███████▏  | 1427/1992 [17:06<08:30,  1.11it/s]

 72%|███████▏  | 1428/1992 [17:06<06:33,  1.43it/s]

 72%|███████▏  | 1429/1992 [17:06<05:03,  1.85it/s]

 72%|███████▏  | 1430/1992 [17:06<04:02,  2.32it/s]

 72%|███████▏  | 1431/1992 [17:07<05:19,  1.76it/s]

 72%|███████▏  | 1432/1992 [17:09<08:35,  1.09it/s]

 72%|███████▏  | 1433/1992 [17:10<10:27,  1.12

 85%|████████▍ | 1693/1992 [21:08<04:35,  1.09it/s]

 85%|████████▌ | 1694/1992 [21:10<05:54,  1.19s/it]

 85%|████████▌ | 1695/1992 [21:10<04:29,  1.10it/s]

 85%|████████▌ | 1696/1992 [21:10<03:22,  1.46it/s]

 85%|████████▌ | 1697/1992 [21:12<05:12,  1.06s/it]

 85%|████████▌ | 1698/1992 [21:12<03:54,  1.25it/s]

 85%|████████▌ | 1699/1992 [21:14<05:58,  1.22s/it]

 85%|████████▌ | 1700/1992 [21:16<07:17,  1.50s/it]

 85%|████████▌ | 1701/1992 [21:17<06:15,  1.29s/it]

 85%|████████▌ | 1702/1992 [21:19<07:21,  1.52s/it]

 85%|████████▌ | 1703/1992 [21:21<07:59,  1.66s/it]

 86%|████████▌ | 1704/1992 [21:23<07:30,  1.57s/it]

 86%|████████▌ | 1705/1992 [21:23<05:29,  1.15s/it]

 86%|████████▌ | 1706/1992 [21:24<05:32,  1.16s/it]

 86%|████████▌ | 1707/1992 [21:25<04:58,  1.05s/it]

 86%|████████▌ | 1708/1992 [21:25<03:59,  1.19it/s]

 86%|████████▌ | 1709/1992 [21:27<04:44,  1.01s/it]

 86%|████████▌ | 1710/1992 [21:27<03:34,  1.31it/s]

 86%|████████▌ | 1711/1992 [21:27<02:46,  1.68

 99%|█████████▉| 1971/1992 [25:10<00:19,  1.09it/s]

 99%|█████████▉| 1972/1992 [25:11<00:19,  1.01it/s]

 99%|█████████▉| 1973/1992 [25:12<00:18,  1.00it/s]

 99%|█████████▉| 1974/1992 [25:12<00:14,  1.25it/s]

 99%|█████████▉| 1975/1992 [25:14<00:16,  1.02it/s]

 99%|█████████▉| 1976/1992 [25:14<00:12,  1.28it/s]

 99%|█████████▉| 1977/1992 [25:15<00:14,  1.02it/s]

 99%|█████████▉| 1978/1992 [25:17<00:15,  1.13s/it]

 99%|█████████▉| 1979/1992 [25:19<00:17,  1.31s/it]

 99%|█████████▉| 1980/1992 [25:19<00:11,  1.00it/s]

 99%|█████████▉| 1981/1992 [25:20<00:10,  1.03it/s]

 99%|█████████▉| 1982/1992 [25:21<00:10,  1.07s/it]

100%|█████████▉| 1983/1992 [25:22<00:07,  1.14it/s]

100%|█████████▉| 1984/1992 [25:23<00:07,  1.02it/s]

100%|█████████▉| 1985/1992 [25:23<00:05,  1.37it/s]

100%|█████████▉| 1986/1992 [25:24<00:04,  1.26it/s]

100%|█████████▉| 1987/1992 [25:25<00:04,  1.21it/s]

100%|█████████▉| 1988/1992 [25:26<00:03,  1.10it/s]

100%|█████████▉| 1989/1992 [25:26<00:02,  1.41